In [ ]:
import pandas as pd

In [ ]:
articles_path = ''
keyword_path = ''
article_id_keyword_mappings_out = ''
selection_df_path = ''

In [ ]:
df_articles = pd.read_csv(articles_path)

# %%
df_articles.head(1)

# %%
df_articles['category'] = df_articles['article_url'].apply(
    lambda x: x.split('/')[3])

In [ ]:
articles_not_in_commentisfree = df_articles[df_articles.category != 'commentisfree']

In [ ]:
df = pd.read_csv(keyword_path)

In [ ]:
df = df[df['keywords'] != 'keywords']

In [ ]:
df = df.drop_duplicates()

In [ ]:
len(df)

In [ ]:
df = df[['id', 'keywords']].reset_index()

In [ ]:
df['article_id'] = df['id']

In [ ]:
df['article_id'] = df['article_id'].astype('int')

In [ ]:
keywords = df.keywords.unique().tolist()

In [ ]:
keyword_to_category = {}
for keyword_string in keywords:
    keyword_list = keyword_string.split(',')
    for k in keyword_list:
        if k not in keyword_to_category:
            keyword_to_category[k] = set()

In [ ]:
articles_not_in_commentisfree = articles_not_in_commentisfree.merge(df, on='article_id')

In [ ]:
for index, row in articles_not_in_commentisfree.iterrows():
    category = row['category']
    keywords = row['keywords'].split(',')
    for k in keywords:
        keyword_to_category[k].add(category)

In [ ]:
df_articles = df_articles.merge(df, on='article_id')

In [ ]:
article_to_category = {}
error_count = 0
for index, row in df_articles.iterrows():
    category = row.category
    if category == 'commentisfree':
        keywords = row.keywords.split(',')
        category_dist = {}
        for k in keywords:
            k_cat = keyword_to_category[k]
            for cat in k_cat:
                if cat in category_dist:
                    category_dist[cat] += 1
                else:
                    category_dist[cat] = 1
        
        max_value = 0
        if len(category_dist) > 1:
            error_count += 1
        for key, value in category_dist.items():
            if value > max_value:
                category = key
        
    article_to_category[row.article_id] = category

In [ ]:
error_count

In [ ]:
len(df_articles)

In [ ]:
df_out = pd.DataFrame.from_dict({'article_id': list(article_to_category.keys()), 'category': list(article_to_category.values())})

In [ ]:
df_out.to_csv(article_id_keyword_mappings_out, index=False)

In [ ]:
comments_df = pd.read_csv(selection_df_path)

In [ ]:
comments_df = comments_df.merge(df_out, on='article_id')

In [ ]:
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
x = comments_df.groupby('category')['comment_id'].count().reset_index()

In [ ]:
x = x.sort_values(by='comment_id', ascending=False)

In [ ]:
def save_to_pdf(filename, ax):
    fig = ax.get_figure()
    fig.savefig(filename, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(3.5,6)})
ax = sns.barplot(y='category', x='comment_id', data=x.head(10), color='dodgerblue')
plt.xticks(rotation=45)
sns.set(rc={'figure.figsize':(5,10)})
ax.set_ylabel('Categories')
ax.set_xlabel('Number of Comments')
save_to_pdf('./plots/guardian_categories_comments_before.pdf', ax)

In [ ]:
comments_df = comments_df.merge(df_articles, on='article_id')